In [1]:
from ecdsa import SigningKey,ellipticcurve,numbertheory
from hashlib import sha256
from random import randint

In [11]:
class server:
    def __init__(self,n):
        self.network=n
        self.value={}
        
    def Rand(self,mod):
        seq=[randint(1000,1000000)%mod for i in range(self.network)]
        self.value[sum(seq)]=seq
        return seq
            
    def RandMul(self,mod):
        a=self.Rand(mod)
        b=self.Rand(mod)
        value_c=sum(a)*sum(b)
        if value_c in self.value:
            return a,b,self.value[value_C]
        seq=[randint(1000,1000000)%mod for i in range(self.network-1)]
        seq.append(value_c-sum(seq)%mod)
        self.value[value_c]=seq
        return a,b,seq
    

class node:
    def __init__(self):
        self.ski=SigningKey.generate()
        self.pki=self.ski.get_verifying_key().pubkey
        self.order=self.ski.curve.order
    
    def preprocessing1(self,ai,bi,c):
        self.ki=ai
        self.inverseki=numbertheory.inverse_mod(c, self.order)*bi%self.order
        return self.pki.generator*self.ki
    
    def preprocessing2_1(self,ci,a_ya,b_xb):
        self.ksi=(ci+a_ya+b_xb)%self.order
    
    def signaturepiece(self,Hash,R):
        return (self.inverseki*Hash)+(R.x()*self.ksi)
        

In [18]:
def keygeneration(n):
    nodelist=[node() for i in range(n)]
    PK= ellipticcurve.Point(None, None, None)
    for Node in nodelist:
        PK=PK+Node.pki.point
    return (PK,nodelist)

def preprocessingone(n,nodelist,server):
    order=nodelist[0].order
    a,b,c=server.RandMul(order)
    cvalue=sum(c)
    R= ellipticcurve.Point(None, None, None)
    for i in range(n):
        R=R+nodelist[i].preprocessing1(a[i],b[i],cvalue)
    return R

def preprocessingtwo(n,nodelist,server):
    order=nodelist[0].order
    (a,b,c)=server.RandMul(order)
    a_y=sum([nodelist[i].ski.privkey.secret_multiplier-a[i] for i in range(n)])
    b_x=sum([nodelist[i].inverseki-b[i] for i in range(n)])
    for i in range(n):
        nodelist[i].preprocessing2_1(c[i],a_y*b[i],b_x*a[i])
    return a_y*b_x

def sigining(message,nodelist,R,bias):
    order=nodelist[0].order
    sign=sum([nodelist[i].signaturepiece(message,R) for i in range(len(nodelist))])+R.x()*bias
    return sign
    
def Verify(message,sign,generator,R,pk):
    return sign*R==R.x()*pk+message*generator



In [19]:
def test(n):
    networksize=n
    testserver=server(networksize)
    Pk,nodelist=keygeneration(networksize)
    R=preprocessingone(networksize,nodelist,testserver)
    bias=preprocessingtwo(networksize,nodelist,testserver)
    for msg in ["Microsoft", "Alibaba", "Google", "Amazon",]:
        Hash=int(sha256(msg.encode("utf-8")).hexdigest(),16)
        signature=sigining(Hash,nodelist,R,bias)
        print(Verify(Hash,signature,nodelist[0].ski.curve.generator,R,Pk))
        

In [20]:
test(2)
test(3)
test(4)
test(5)

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
